In [ ]:
#This notebook is for plotting of PIPS data from the real-time script's netCDF file dumps and merging with netCDF
#files produced from the onboard card data if necessary
%matplotlib notebook
import os
import pandas as pd
import xarray as xr
import numpy as np
import time
import glob
import matplotlib.pyplot as plt
from pyPIPS import thermolib as thermo
from pyPIPS import timemodule as tm
from datetime import datetime, timedelta
import matplotlib.dates as dates
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid, make_axes_locatable, host_subplot
import pyPIPS.plotmodule as pm
from pyPIPS.PIPS import avg_diameter, avg_fall_bins, max_diameter, \
    min_diameter, min_fall_bins, diameter_edges, fall_bins_edges
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
# Function definitions. 

def get_files(file_list, starttime, endtime, ftype='onesec'):
    ''' this seems like a janky way to do it, but is actually 3x faster than
        making a loop of files.'''

    len_prefix = 8 + len(ftype)

    day_str = [(starttime + timedelta(days=i)).strftime("%Y%m%d")
               for i in range((endtime - starttime).days + 1)]

    # find all PIPS files with days between starttime and endtime
    file_list = [file_name for file_name in file_list if any(day in file_name for day in day_str)]
    # file_list = [f for subf in file_list for f in subf]  # flatten list in case of multiple days

    if file_list:
        # sort files by date, then find nearest indices for all the dates, and loop over that
        sorted_files = sorted(file_list,
                              key=lambda f: datetime.strptime(f[len_prefix:len_prefix + 14],
                                                              '%Y%m%d%H%M%S'))
        starttimes = [datetime.strptime(f[len_prefix:len_prefix + 14], '%Y%m%d%H%M%S')
                      for f in sorted_files]
        endtimes = [datetime.strptime(f[len_prefix + 15:len_prefix + 29], '%Y%m%d%H%M%S')
                    for f in sorted_files]
        _, idx1 = min((abs(val - starttime), idx) for (idx, val) in enumerate(starttimes))
        _, idx2 = min((abs(val - endtime), idx) for (idx, val) in enumerate(endtimes))
        file_list = sorted_files[idx1:idx2 + 1]

    return file_list


In [ ]:
# Set up directories reading the data and plotting
# base_output_dir = '/Users/dawson29/sshfs_mounts/depot/data/Projects/TriPIPS/webdata/'
deployment_name = 'IOP2_030323'

base_realtime_input_dir = '/Users/dawson29/Dropbox/Projects/PERiLS/PIPS_data/2023_realtime'
realtime_netcdf_input_dir = os.path.join(base_realtime_input_dir, deployment_name)

base_card_input_dir = '/Users/dawson29/Dropbox/PIPS_data/2023/'
card_netcdf_input_dir = os.path.join(base_card_input_dir, deployment_name, 'netcdf')

plot_output_dir = os.path.join(base_realtime_input_dir, 'plots')

if not os.path.exists(plot_output_dir):
    os.makedirs(plot_output_dir)


In [ ]:
# Set up the PIPS name, dates and times to read and plot

PIPS_name = 'PIPS1A'

# PIPS1A
starttime_stamp = '20230303081700'
endtime_stamp = '20230303115100'

# PIPS2A
# starttime_stamp = '20230303084100'
# endtime_stamp = '20230303111800'


starttime_dt = datetime.strptime(starttime_stamp, tm.timefmt3)
endtime_dt = datetime.strptime(endtime_stamp, tm.timefmt3)

file_path_list_onePIPS = glob.glob(realtime_netcdf_input_dir + f'/*{PIPS_name}*nc')
file_list_onePIPS = [os.path.basename(file_path) for file_path in file_path_list_onePIPS]

file_list_onesec = [file_name for file_name in file_list_onePIPS if 'onesec' in file_name]
file_list_onesec = [file_name for file_name in file_list_onesec if 'current' not in file_name]
file_list_onesec = get_files(file_list_onesec, starttime_dt, endtime_dt)
file_path_list_onesec = [os.path.join(realtime_netcdf_input_dir, file_name) for file_name in file_list_onesec]


file_list_ND = [file_name for file_name in file_list_onePIPS if 'ND' in file_name]
file_list_ND = [file_name for file_name in file_list_ND if 'current' not in file_name]
file_list_ND = get_files(file_list_ND, starttime_dt, endtime_dt, ftype='ND')
file_path_list_ND = [os.path.join(realtime_netcdf_input_dir, file_name) for file_name in file_list_ND]

file_list_spectrum = [file_name for file_name in file_list_onePIPS if 'spectrum' in file_name]
file_list_spectrum = [file_name for file_name in file_list_spectrum if 'current' not in file_name]
file_list_spectrum = get_files(file_list_spectrum, starttime_dt, endtime_dt, ftype='spectrum')
file_path_list_spectrum = [os.path.join(realtime_netcdf_input_dir, file_name) for file_name in file_list_spectrum]

file_list_telegram = [file_name for file_name in file_list_onePIPS if 'telegram' in file_name]
file_list_telegram = [file_name for file_name in file_list_telegram if 'current' not in file_name]
file_list_telegram = get_files(file_list_telegram, starttime_dt, endtime_dt, ftype='telegram')
file_path_list_telegram = [os.path.join(realtime_netcdf_input_dir, file_name) for file_name in file_list_telegram]

# datetime_stamp = '20220505210000'
# ND_file = 'ND_{}.nc'.format(datetime_stamp)
# ND_path = os.path.join(netcdf_input_dir, ND_file)
# ND_ds = xr.load_dataset(ND_path)

# conv_file = 'onesec_{}.nc'.format(datetime_stamp)
# conv_path = os.path.join(netcdf_input_dir, conv_file)
# conv_ds = xr.load_dataset(conv_path)


In [ ]:
# Now read in netCDF files
onesec_ds = xr.open_mfdataset(file_path_list_onesec, combine='nested', concat_dim='logger_datetime')
onesec_ds = onesec_ds.rename({'logger_datetime': 'time'})
onesec_ds = onesec_ds.drop_duplicates('time')
ND_ds = xr.open_mfdataset(file_path_list_ND, combine='nested', concat_dim='time')
ND_ds = ND_ds.drop_duplicates('time')
ND_ds = ND_ds.rename_dims({'diameter': 'diameter_bin'})
spectrum_ds = xr.open_mfdataset(file_path_list_spectrum, combine='nested', concat_dim='time')
spectrum_ds = spectrum_ds.drop_duplicates('time')
spectrum_ds = spectrum_ds.rename_dims({'diameter': 'diameter_bin', 'velocity': 'velocity_bin'})
telegram_ds = xr.open_mfdataset(file_path_list_telegram, combine='nested', concat_dim='index')
telegram_ds = telegram_ds.rename({'index': 'time'})
telegram_ds = telegram_ds.drop_duplicates('time')

In [ ]:
# Read netCDF files from card
parsivel_combined_card_filename = f'parsivel_combined_{deployment_name}_{PIPS_name}_10s.nc'
parsivel_combined_card_path = os.path.join(card_netcdf_input_dir, parsivel_combined_card_filename)

onesec_card_filename = f'conventional_raw_{deployment_name}_{PIPS_name}.nc'
onesec_card_path = os.path.join(card_netcdf_input_dir, onesec_card_filename)

parsivel_combined_card_ds = xr.open_dataset(parsivel_combined_card_path)
onesec_card_ds = xr.open_dataset(onesec_card_path)

In [ ]:
# Correct times from from the real-time files to match the GPS times. This is already done for the times
# from the card-derived netCDF files

# Get first good GPS time in file

first_good_GPS_time = onesec_ds.where(onesec_ds['GPS_status'] == 'A', drop=True).isel(time=0)

logger_datetime = first_good_GPS_time['time'].values
logger_datetime = pd.to_datetime(logger_datetime).to_pydatetime()

GPS_date = str(first_good_GPS_time['GPS_date'].values)
GPS_time = str(first_good_GPS_time['GPS_time'].values)

# Next, construct datetime object from GPS info
# Construct datetime object
gyear = int('20' + GPS_date[4:])
gmonth = int(GPS_date[2:4])
gday = int(GPS_date[:2])
ghour = int(GPS_time[:2])
gmin = int(GPS_time[2:4])
gsec = int(GPS_time[4:6])

GPS_datetime = datetime(gyear, gmonth, gday, ghour, gmin, gsec)
GPS_offset = GPS_datetime - logger_datetime
print('GPS time: {}, Logger time: {}'.format(GPS_datetime.ctime(),
                                             logger_datetime.ctime()))
print('GPS Offset: {}'.format(str(GPS_offset)))

# print(onesec_card_ds.where(onesec_card_ds['GPS_status'] == 'A', drop=True).isel(time=0))

In [ ]:
old_times = pd.to_datetime(onesec_ds['time']).to_pydatetime()
new_times = old_times + GPS_offset
onesec_ds = onesec_ds.assign_coords({'time': new_times})

In [ ]:
old_ND_times = pd.to_datetime(ND_ds['time']).to_pydatetime()
new_times = old_ND_times + GPS_offset
ND_ds = ND_ds.assign_coords({'time': new_times})

old_spectrum_times = pd.to_datetime(spectrum_ds['time']).to_pydatetime()
new_times = old_spectrum_times + GPS_offset
spectrum_ds = spectrum_ds.assign_coords({'time': new_times})

old_telegram_times = pd.to_datetime(telegram_ds['time']).to_pydatetime()
new_times = old_telegram_times + GPS_offset
telegram_ds = telegram_ds.assign_coords({'time': new_times})

In [ ]:
# Now, figure out what times are missing from the data for each source

# First find earliest start and latest end times
tensec_card_starttime = parsivel_combined_card_ds['time'][0].values
tensec_card_endtime = parsivel_combined_card_ds['time'][-1].values
spectrum_starttime = spectrum_ds['time'][0].values
spectrum_endtime = spectrum_ds['time'][-1].values
telegram_starttime = telegram_ds['time'][0].values
telegram_endtime = telegram_ds['time'][-1].values
ND_starttime = ND_ds['time'][0].values
ND_endtime = ND_ds['time'][-1].values

starttime_tensec = min(tensec_card_starttime, spectrum_starttime, telegram_starttime, ND_starttime)
endtime_tensec = max(tensec_card_endtime, spectrum_endtime, telegram_endtime, ND_endtime)

print(starttime_tensec, endtime_tensec)

# Then, create a new index of all times at 10-s intervals between the two

all_tensec_times = xr.date_range(starttime_tensec, endtime_tensec, freq='10S')
print(all_tensec_times)
print(all_tensec_times[0], all_tensec_times[-1])

print(ND_ds['time'])
print(spectrum_ds['time'])
print(parsivel_combined_card_ds['time'])

In [ ]:
# Now reindex all datasets with the full set of times

ND_ds_full = ND_ds.reindex({'time': all_tensec_times})
print(ND_ds_full)

In [ ]:
# Set up dictionaries to control plotting parameters

dateformat = '%H:%M'

# Temperature and dewpoint
temp_dewp_ax_params = {
    'majorxlocator': dates.MinuteLocator(byminute=[0, 15, 30, 45], interval=1), 
    'majorxformatter': dates.DateFormatter(dateformat),
    'minorxlocator': dates.MinuteLocator(byminute=[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55], interval=1),
    'axeslimits': [None, (-5., 35.)],
    'axeslabels': ['Time (H:M) UTC', r'Temperature ($^{\circ}$C)']
}

# Wind speed and direction
windspd_ax_params = {
    'majorxlocator': dates.MinuteLocator(byminute=[0, 15, 30, 45], interval=1), 
    'majorxformatter': dates.DateFormatter(dateformat),
    'minorxlocator': dates.MinuteLocator(byminute=[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55], interval=1),
    'axeslimits': [None, [0.0, 25.0]],
    'axeslabels': ['Time (H:M) UTC', r'wind speed (m s$^{-1}$)']
}

winddir_ax_params = {
    'majorylocator': ticker.MultipleLocator(45.),
    'axeslimits': [None, [0.0, 360.0]],
    'axeslabels': [None, r'Wind direction ($^{\circ}$C)']
}

pressure_ax_params = {
    'majorylocator': ticker.MultipleLocator(5.),
    'axeslimits': [None, [940., 980.]],
    'axeslabels': [None, r'Pressure (hPa)']
}



# Number concentration
log_ND_params = {
    'type': 'pcolor', 
    'vlimits': [-1.0, 3.0],
    'clabel': r'log[N ($m^{-3} mm^{-1}$)]'
}

log_ND_ax_params = {
    'majorxlocator': dates.MinuteLocator(byminute=[0, 15, 30, 45], interval=1), 
    'majorxformatter': dates.DateFormatter(dateformat),
    'minorxlocator': dates.MinuteLocator(byminute=[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55], interval=1),
    'axeslimits': [None, [0.0, 9.0]],
    'majorylocator': ticker.MultipleLocator(base=0.25),
    'axeslabels': [None, 'D (mm)']
}

In [ ]:
# Quick DSD meteogram
fig, ax = plt.subplots(figsize=(10, 6))
plottimes_tmp = ND_ds_full['time'].to_index().to_pydatetime()
# Prepend an additional at the beginning of the array so that pcolor sees this as the
# edges of the DSD intervals.
plottimes = np.insert(plottimes_tmp, 0, plottimes_tmp[0] - timedelta(seconds=10))
plottimes = [plottimes]

ND_da = ND_ds_full['ND']

ND_arr = ND_da.values.T
logND_arr = np.ma.log10(ND_arr)
fields_to_plot = [logND_arr]
field_parameters = [log_ND_params]
ax = pm.plotmeteogram(ax, plottimes, fields_to_plot, field_parameters,
                      yvals=[diameter_edges] * len(fields_to_plot))
ax, = pm.set_meteogram_axes([ax], [log_ND_ax_params])

In [ ]:
# Quick DSD meteogram for card data
fig, ax = plt.subplots(figsize=(10, 6))
plottimes_tmp = parsivel_combined_card_ds['time'].to_index().to_pydatetime()
# Prepend an additional at the beginning of the array so that pcolor sees this as the
# edges of the DSD intervals.
plottimes = np.insert(plottimes_tmp, 0, plottimes_tmp[0] - timedelta(seconds=10))
plottimes = [plottimes]

ND_da_card = parsivel_combined_card_ds['ND']

ND_arr = ND_da_card.values.T
logND_arr = np.ma.log10(ND_arr)
fields_to_plot = [logND_arr]
field_parameters = [log_ND_params]
ax = pm.plotmeteogram(ax, plottimes, fields_to_plot, field_parameters,
                      yvals=[diameter_edges] * len(fields_to_plot))
ax, = pm.set_meteogram_axes([ax], [log_ND_ax_params])

In [ ]:
# Attempt to merge the data to fill in gaps
ND_da_new = ND_da.combine_first(ND_da_card)

In [ ]:
print(ND_da_new)
print(ND_da)
print(ND_da_card)

In [ ]:
ND_da_new.equals(ND_da)

In [ ]:
print(ND_da_new.drop_duplicates('time'))

In [ ]:
print(ND_da_new.sel(time=slice('2023-03-03 T10:05', '2023-03-03 T10:20')).coords['time'])

In [ ]:
# Quick DSD meteogram for merged data
fig, ax = plt.subplots(figsize=(10, 6))
plottimes_tmp = ND_da_new.coords['time'].to_index().to_pydatetime()
# Prepend an additional at the beginning of the array so that pcolor sees this as the
# edges of the DSD intervals.
plottimes = np.insert(plottimes_tmp, 0, plottimes_tmp[0] - timedelta(seconds=10))
plottimes = [plottimes]

ND_arr = ND_da_new.values.T
logND_arr = np.ma.log10(ND_arr)
fields_to_plot = [logND_arr]
field_parameters = [log_ND_params]
ax = pm.plotmeteogram(ax, plottimes, fields_to_plot, field_parameters,
                      yvals=[diameter_edges] * len(fields_to_plot))
ax, = pm.set_meteogram_axes([ax], [log_ND_ax_params])

In [ ]:
# Quick DSD plot
fig_dsd, ax_dsd = plt.subplots(figsize=(10, 10))
time_to_plot = '20220505213500'
datetime_to_plot = datetime.strptime(time_to_plot, tm.timefmt3)
datetime_minus_5min = datetime_to_plot - timedelta(minutes=5)
ND_avg_da = ND_da.sel(time=slice(datetime_minus_5min, datetime_to_plot)).mean(dim='time')

ND_avg = ND_avg_da.values
ax_dsd.set_title('DSD for time period {0} to {1}'.format(
    datetime_minus_5min.strftime(tm.timefmt2),
    datetime_to_plot.strftime(tm.timefmt2)))
ax_dsd.bar(min_diameter, ND_avg * 1000.0, max_diameter - min_diameter, 10.**2., align='edge',
           log=True, color='tan', edgecolor='k')
ax_dsd.set_xlim(0.0, 3.0)
ax_dsd.set_ylim(10.**2., 10.**6.5)
ax_dsd.set_xlabel('D (mm)')
ax_dsd.set_ylabel(r'N(D) $(m^{-4})$')

In [ ]:
fig_t_td, ax_t_td = plt.subplots(figsize=(10, 6))
plottimes_onesec = [conv_ds['index'].to_index().to_pydatetime()]
# Temperature and Dewpoint
Tmin = np.nanmin(conv_ds['Dewpoint'].values)
Tmax = np.nanmax(conv_ds['SlowTemp'].values)
fields_to_plot_onesec = [conv_ds['SlowTemp'].values, conv_ds['Dewpoint'].values]
temp_params = pm.temp_params.copy()
dewpoint_params = pm.dewpoint_params.copy()
temp_params['plotmin'] = Tmin - 5.0
dewpoint_params['plotmin'] = Tmin - 5.0
field_parameters_onesec = [temp_params, dewpoint_params]
ax_t_td = pm.plotmeteogram(
    ax_t_td,
    plottimes_onesec,
    fields_to_plot_onesec,
    field_parameters_onesec)
temp_dewp_ax_params['axeslimits'] = [[plottimes_onesec[0][0], plottimes_onesec[0][-1]],
                                     [Tmin - 5.0, Tmax + 5.0]]
ax_t_td, = pm.set_meteogram_axes([ax_t_td], [temp_dewp_ax_params])

In [ ]:
#%%time
#for i in range(numiter):
while True:
    ax.clear()
    ax_vd.clear()
    ax_t_td.clear()
    ax_windspd.clear()
    ax_winddir.clear()
    ax_pressure.clear()
    onesec_new_df = scrape_tripips_onesec_data(url_onesec, numrecords=numrecords_append_onesec)
    onesec_new_df['Dewpoint'] = thermo.calTdfromRH(onesec_new_df['Pressure'] * 100., onesec_new_df['SlowTemp'] + 273.15, 
                                     onesec_new_df['RH'] / 100.) - 273.15
    # Append new data onto onesec_df
    onesec_df = onesec_df.append(onesec_new_df)
    # Drop duplicate timestamps
    onesec_df = onesec_df[~onesec_df.index.duplicated(keep='first')]
    # Drop records older than desired interval
    last_timestamp_onesec = onesec_df.index[-1]
    oldest_timestamp_onesec = last_timestamp_onesec-keep_data_for_ts
    onesec_df = onesec_df.loc[oldest_timestamp_onesec:]
    # Dump onesec dataframe to netCDF file (via xarray)
    netcdf_filename = 'onesec_{}_{}.nc'.format(oldest_timestamp_onesec.strftime('%Y%m%d%H%M%S'), 
                                               last_timestamp_onesec.strftime('%Y%m%d%H%M%S' ))
    netcdf_path = os.path.join(netcdf_output_dir, netcdf_filename)
    onesec_df.to_xarray().to_netcdf(netcdf_path)
    plottimes_onesec = [onesec_df.index.to_pydatetime()]
    # Temperature and Dewpoint
    fields_to_plot_onesec = [onesec_df['SlowTemp'].values, onesec_df['Dewpoint'].values]
    field_parameters_onesec = [pm.temp_params, pm.dewpoint_params]
    ax_t_td = pm.plotmeteogram(ax_t_td, plottimes_onesec, fields_to_plot_onesec, field_parameters_onesec)
    temp_dewp_ax_params['axeslimits'][0] = (plottimes_onesec[0][0], plottimes_onesec[0][-1]) 
    ax_t_td, = pm.set_meteogram_axes([ax_t_td], [temp_dewp_ax_params])
    # Wind speed and direction
    ax_windspd = pm.plotmeteogram(ax_windspd, plottimes_onesec, [onesec_df['WS_ms'].values], [pm.windspeed_params])
    ax_winddir = pm.plotmeteogram(ax_winddir, plottimes_onesec, [onesec_df['WindDir'].values], [pm.winddir_params])
    windspd_ax_params['axeslimits'][0] = (plottimes_onesec[0][0], plottimes_onesec[0][-1])
    winddir_ax_params['axeslimits'][0] = (plottimes_onesec[0][0], plottimes_onesec[0][-1])
    ax_windspd, ax_winddir = pm.set_meteogram_axes([ax_windspd, ax_winddir], [windspd_ax_params, winddir_ax_params])
    # Pressure
    pmin = np.nanmin(onesec_df['Pressure'].values)
    pmax = np.nanmax(onesec_df['Pressure'].values)
    pressure_ax_params['axeslimits'] = [None, [pmin - 2.5, pmax + 2.5]]
    fields_to_plot_press = [onesec_df['Pressure'].values]
    field_parameters_press = [pm.pressure_params]
    ax_pressure = pm.plotmeteogram(ax_pressure, plottimes_onesec, fields_to_plot_press, field_parameters_press)
    ax_pressure, = pm.set_meteogram_axes([ax_pressure], [pressure_ax_params])
    

    # DSD plots
    telegram_new_df, spectrum_new_da = scrape_tripips_tensec_data(url_tensec, numrecords=numrecords_append_tensec)
    ND_new_da = calc_ND_da(spectrum_new_da)
    # Append new data onto the data array
    ND_da = xr.concat([ND_da, ND_new_da], dim='time')
    # Drop duplicate timestamps
    ND_da = ND_da.groupby('time').first()
    # onesec_df = onesec_df[~onesec_df.index.duplicated(keep='first')]
    # Drop records older than desired interval
    last_timestamp = ND_da['time'].to_index()[-1]
    spectrum = spectrum_new_da.loc[last_timestamp]
    # print(last_timestamp)
    oldest_timestamp = last_timestamp-keep_data_for_ts
    ND_da = ND_da.loc[oldest_timestamp:]
    # Dump ND_da to netcdf file
    
    netcdf_filename = 'ND_{}_{}.nc'.format(oldest_timestamp.strftime('%Y%m%d%H%M%S'), 
                                       last_timestamp.strftime('%Y%m%d%H%M%S' ))
    netcdf_path = os.path.join(netcdf_output_dir, netcdf_filename)
    ND_da.to_dataset(name='ND').to_netcdf(netcdf_path)
                                
    plottimes = [ND_da['time'].to_index().to_pydatetime()]
    ND_arr = ND_da.values.T
    logND_arr = np.ma.log10(ND_arr)
    fields_to_plot = [logND_arr]
    field_parameters = [log_ND_params]
    ax = pm.plotmeteogram(ax, plottimes, fields_to_plot, field_parameters, 
                          yvals=[min_diameter] * len(fields_to_plot))
    ax, = pm.set_meteogram_axes([ax], [log_ND_ax_params])
    ax_vd.set_title('Fall speed vs. diameter for time {0}'.format(last_timestamp.strftime(tm.timefmt2)))
    countsplot = np.ma.masked_where(spectrum.values <= 0, spectrum)
    C = ax_vd.pcolor(min_diameter, min_fall_bins, countsplot, vmin=1, vmax=50, edgecolors='w')
    divider = make_axes_locatable(ax_vd)
    cax = divider.append_axes("right", size="5%")
    cb = fig_vd.colorbar(C, cax=cax, orientation='vertical')
    ax_vd.set_xlim(0.0, 10.0)
    ax_vd.xaxis.set_major_locator(ticker.MultipleLocator(2.0))
    ax_vd.set_xlabel('diameter (mm)')
    ax_vd.set_ylim(0.0, 10.0)
    ax_vd.yaxis.set_major_locator(ticker.MultipleLocator(1.0))
    ax_vd.set_ylabel('fall speed (m/s)')
    
    display.display(fig)
    display.display(fig_vd)
    display.display(fig_t_td)
    display.display(fig_wind)
    display.display(fig_pressure)
    display.clear_output(wait=True)
    # fig.canvas.draw()
    fig.savefig(os.path.join(image_output_dir, 'logND_current.png'), dpi=300)
    fig_vd.savefig(os.path.join(image_output_dir, 'VD_current.png'), dpi=300)
    fig_t_td.savefig(os.path.join(image_output_dir, 'T_Td_current.png'), dpi=300)
    fig_wind.savefig(os.path.join(image_output_dir, 'wind_current.png'), dpi=300)
    fig_pressure.savefig(os.path.join(image_output_dir, 'pressure.png'), dpi=300)
    
    # plt.pause(0.01)
    # Sleep for the desired interval. This may not be perfectly accurate
    time.sleep(plot_update_interval)